# 1. Preprocess Training Data

*Version: 2022-04-19*

Prerequisite: necessary corpora have been downloaded and stored in `lexnlp/ml/catalog/data/`.

Algorithm:

```text
For each corpus:
    Extract all `.txt` files.
    For each text file:
        Get sentences until their combined lengths sum to some limit L.
        Normalize each sentence.
        Lemmatize each sentence.
        Tokenize each sentence.
        Combine token lists into one string.
        Write each string as a single line into a combined text file.
```

---

**Imports**

In [1]:
# standard library
import csv
from pathlib import Path
from tarfile import TarFile
from tempfile import TemporaryDirectory
from typing import Final, List, Set, Tuple
from concurrent.futures import ProcessPoolExecutor

# third-party libraries
from psutil import Process
from tqdm.notebook import tqdm

# LexNLP
from lexnlp.ml.catalog import get_path_from_catalog
from lexnlp.ml.normalizers import Normalizer
from lexnlp.ml.sklearn_transformers import TransformerPreprocessor
from lexnlp.extract.en.addresses.addresses import get_address_annotations
from lexnlp.extract.en.amounts import get_amount_annotations
from lexnlp.extract.en.dates import get_date_annotations
from lexnlp.extract.en.percents import get_percent_annotations
from lexnlp.extract.en.ratios import get_ratio_annotations

/home/aparsons/lexpredict-contraxsuite-core/venv_lexpredict_contraxsuite_core/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.tree.tree module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.tree. Anything that cannot be imported from sklearn.tree is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/aparsons/lexpredict-contraxsuite-core/venv_lexpredict_contraxsuite_core/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.19.1 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


---

## Constants, Objects, and Functions

In [2]:
# directory paths
PATH_PREPROCESSED: Final[Path] = Path('./preprocessed/')
PATH_PREPROCESSED.mkdir(exist_ok=True)

# number of processors to use for multiprocessing
MAX_WORKERS: Final[int] = (len(Process().cpu_affinity()) - 1) or 1

In [3]:
TAGS: Set[str] = {

    # Atticus CUAD v1 contracts
    # 'corpus/atticus-cuad-v1-plaintext/0.1',

    # labeled contract types
    'corpus/contract-types/0.1',
}

In [4]:
def kwarg_wrapper_get_amount_annotations(text):
    yield from get_amount_annotations(text=text, extended_sources=False)

NORMALIZER: Final[Normalizer] = Normalizer(
    normalizations=(
        (get_address_annotations, '__ADDRESS__'),
        (get_ratio_annotations, '__RATIO__'),
        (get_date_annotations, '__DATE__'),
        (get_percent_annotations, '__PERCENT__'),
        (kwarg_wrapper_get_amount_annotations, '__AMOUNT__'),
    ),
)

In [5]:
TRANSFORMER_PREPROCESSOR: TransformerPreprocessor = TransformerPreprocessor(
    normalizer=NORMALIZER,
)

In [6]:
def _wrapper_preprocess(label_and_text: Tuple[str, str]) -> Tuple[str, str]:
    return label_and_text[0], TRANSFORMER_PREPROCESSOR.preprocess(label_and_text[1])

def preprocess_corpus(path_corpus: Path) -> Path:
    """
    Extracts and reads all `.txt` files, saving their contents in a list of strings.
    Then preprocesses each text and saves to a different list of strings.
    Each string from the second list is written as a line in the output file.

    Args:
        path_corpus (Path):
            A Path to a corpus tarfile containing text files.

    Returns:
        A Path to the output file.
    """
    texts: List[Tuple[str, str]] = []
    with TemporaryDirectory() as temporary_directory:
        with TarFile.open(name=path_corpus, mode=f'r|{path_corpus.suffix.split(".")[-1]}') as tar_file:
            tar_file.extractall(path=temporary_directory)
            text_files: Tuple[Path] = tuple(Path(temporary_directory).rglob('*.txt'))
            for text_file in text_files:
                label: str = text_file.parent.parts[-1]
                with text_file.open('r') as f:
                    try:
                        text: str = f.read()
                        texts.append((label, text))
                    except UnicodeDecodeError as unicode_decode_error:
                        print(f'...{unicode_decode_error} @ {text_file}')

    with ProcessPoolExecutor(max_workers=MAX_WORKERS) as process_pool_executor:
        documents: List[str] = list(tqdm(
            process_pool_executor.map(_wrapper_preprocess, texts),
            total=len(texts)
        ))

    directory: Path = PATH_PREPROCESSED / tag
    directory.mkdir(parents=True, exist_ok=True)
    path_output_file: Path = directory / f'{path_corpus.name}.csv'
    with path_output_file.open('w') as f:
        csv_writer = csv.writer(f)
        csv_writer.writerow(('label', 'text'))
        for label_and_text in documents:
            csv_writer.writerow(label_and_text)
    return path_output_file

---

## Preprocess Corpora

In [7]:
for tag in TAGS:
    path_corpus: Path = get_path_from_catalog(tag=tag)
    print('Preprocessing', tag)
    preprocess_corpus(path_corpus=path_corpus)

Preprocessing corpus/contract-types/0.1


  0%|          | 0/2387 [00:00<?, ?it/s]